In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
# Load the processed data
processed_data = pd.read_csv('../data/processed_financial_data.csv')
with open('../model/feature_info.json', 'r') as f:
    feature_info = json.load(f)


In [ ]:
class FinancialAttentionModel:
    def __init__(self, n_features, n_heads=8, d_model=128, dropout_rate=0.1):
        self.n_features = n_features
        self.n_heads = n_heads
        self.d_model = d_model
        self.dropout_rate = dropout_rate
        
    def positional_encoding(self, seq_len, d_model):
        """Create positional encoding for sequence data"""
        position = np.arange(seq_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        
        pos_encoding = np.zeros((seq_len, d_model))
        pos_encoding[:, 0::2] = np.sin(position * div_term)
        pos_encoding[:, 1::2] = np.cos(position * div_term)
        
        return pos_encoding
    
    def multi_head_attention_block(self, inputs, name_prefix="attention"):
        """Multi-head attention block with residual connection"""
        # Multi-head self-attention
        attention_output = layers.MultiHeadAttention(
            num_heads=self.n_heads,
            key_dim=self.d_model // self.n_heads,
            dropout=self.dropout_rate,
            name=f"{name_prefix}_mha"
        )(inputs, inputs)
        
        # Add & Norm
        attention_output = layers.Dropout(self.dropout_rate)(attention_output)
        attention_output = layers.Add(name=f"{name_prefix}_add1")([inputs, attention_output])
        attention_output = layers.LayerNormalization(name=f"{name_prefix}_norm1")(attention_output)
        
        # Feed-forward network
        ffn_output = layers.Dense(self.d_model * 4, activation='relu', name=f"{name_prefix}_ffn1")(attention_output)
        ffn_output = layers.Dropout(self.dropout_rate)(ffn_output)
        ffn_output = layers.Dense(self.d_model, name=f"{name_prefix}_ffn2")(ffn_output)
        
        # Add & Norm
        ffn_output = layers.Dropout(self.dropout_rate)(ffn_output)
        ffn_output = layers.Add(name=f"{name_prefix}_add2")([attention_output, ffn_output])
        output = layers.LayerNormalization(name=f"{name_prefix}_norm2")(ffn_output)
        
        return output
    
    def build_classification_model(self, task_name="savings_prediction"):
        """Build attention model for binary classification"""
        # Input layer - treating each feature as a sequence element
        inputs = layers.Input(shape=(self.n_features,), name="financial_features")
        
        # Reshape for attention mechanism (batch_size, sequence_length=1, features)
        x = layers.Reshape((1, self.n_features))(inputs)
        
        # Project to d_model dimensions
        x = layers.Dense(self.d_model, name="input_projection")(x)
        
        # Add positional encoding (even though sequence length is 1, useful for extensibility)
        pos_encoding = self.positional_encoding(1, self.d_model)
        x = x + pos_encoding
        
        # Multiple attention blocks
        x = self.multi_head_attention_block(x, "attention_block_1")
        x = self.multi_head_attention_block(x, "attention_block_2")
        
        # Global average pooling
        x = layers.GlobalAveragePooling1D()(x)
        
        # Final classification layers
        x = layers.Dense(256, activation='relu', name="dense_1")(x)
        x = layers.Dropout(self.dropout_rate)(x)
        x = layers.Dense(128, activation='relu', name="dense_2")(x)
        x = layers.Dropout(self.dropout_rate)(x)
        x = layers.Dense(64, activation='relu', name="dense_3")(x)
        
        # Output layer
        outputs = layers.Dense(1, activation='sigmoid', name=task_name)(x)
        
        model = Model(inputs=inputs, outputs=outputs, name=f"financial_{task_name}_model")
        
        return model
    
    def build_multi_task_model(self):
        """Build multi-task attention model for multiple predictions"""
        # Input layer
        inputs = layers.Input(shape=(self.n_features,), name="financial_features")
        
        # Reshape for attention mechanism
        x = layers.Reshape((1, self.n_features))(inputs)
        
        # Project to d_model dimensions
        x = layers.Dense(self.d_model, name="input_projection")(x)
        
        # Add positional encoding
        pos_encoding = self.positional_encoding(1, self.d_model)
        x = x + pos_encoding
        
        # Shared attention blocks
        shared_features = self.multi_head_attention_block(x, "shared_attention_1")
        shared_features = self.multi_head_attention_block(shared_features, "shared_attention_2")
        
        # Global pooling
        pooled_features = layers.GlobalAveragePooling1D()(shared_features)
        
        # Task-specific branches
        # 1. Savings Achievement Classification
        savings_branch = layers.Dense(128, activation='relu', name="savings_dense_1")(pooled_features)
        savings_branch = layers.Dropout(self.dropout_rate)(savings_branch)
        savings_branch = layers.Dense(64, activation='relu', name="savings_dense_2")(savings_branch)
        savings_output = layers.Dense(1, activation='sigmoid', name="can_achieve_savings")(savings_branch)
        
        # 2. Savings Amount Regression
        amount_branch = layers.Dense(128, activation='relu', name="amount_dense_1")(pooled_features)
        amount_branch = layers.Dropout(self.dropout_rate)(amount_branch)
        amount_branch = layers.Dense(64, activation='relu', name="amount_dense_2")(amount_branch)
        amount_output = layers.Dense(1, name="savings_amount")(amount_branch)
        
        # 3. Financial Risk Classification
        risk_branch = layers.Dense(128, activation='relu', name="risk_dense_1")(pooled_features)
        risk_branch = layers.Dropout(self.dropout_rate)(risk_branch)
        risk_branch = layers.Dense(64, activation='relu', name="risk_dense_2")(risk_branch)
        risk_output = layers.Dense(1, activation='sigmoid', name="financial_risk")(risk_branch)
        
        model = Model(
            inputs=inputs, 
            outputs=[savings_output, amount_output, risk_output],
            name="financial_multi_task_model"
        )
        
        return model
    
    def build_regression_model(self, task_name="savings_amount"):
        """Build attention model for regression tasks"""
        inputs = layers.Input(shape=(self.n_features,), name="financial_features")
        
        x = layers.Reshape((1, self.n_features))(inputs)
        x = layers.Dense(self.d_model, name="input_projection")(x)
        
        pos_encoding = self.positional_encoding(1, self.d_model)
        x = x + pos_encoding
        
        x = self.multi_head_attention_block(x, "attention_block_1")
        x = self.multi_head_attention_block(x, "attention_block_2")
        
        x = layers.GlobalAveragePooling1D()(x)
        
        x = layers.Dense(256, activation='relu', name="dense_1")(x)
        x = layers.Dropout(self.dropout_rate)(x)
        x = layers.Dense(128, activation='relu', name="dense_2")(x)
        x = layers.Dropout(self.dropout_rate)(x)
        x = layers.Dense(64, activation='relu', name="dense_3")(x)
        
        outputs = layers.Dense(1, name=task_name)(x)
        
        model = Model(inputs=inputs, outputs=outputs, name=f"financial_{task_name}_model")
        
        return model

In [ ]:
def prepare_data_for_training(processed_data, feature_info):
    """Prepare data for training attention models"""
    
    # Get feature columns (exclude target variables)
    feature_columns = feature_info['numerical_features'] + feature_info['categorical_features']
    
    # Prepare features
    X = processed_data[feature_columns].values
    
    # Prepare targets
    targets = {}
    if 'Can_Achieve_Savings' in processed_data.columns:
        targets['can_achieve_savings'] = processed_data['Can_Achieve_Savings'].values
    if 'Desired_Savings' in processed_data.columns:
        targets['savings_amount'] = processed_data['Desired_Savings'].values
    if 'Savings_Gap' in processed_data.columns:
        # Create financial risk based on savings gap
        targets['financial_risk'] = (processed_data['Savings_Gap'] > processed_data['Savings_Gap'].quantile(0.7)).astype(int).values
    
    print(f"Features shape: {X.shape}")
    print(f"Available targets: {list(targets.keys())}")
    for name, target in targets.items():
        print(f"{name} distribution: {np.bincount(target) if target.dtype == int else 'continuous'}")
    
    return X, targets, feature_columns

# Prepare the data
X, targets, feature_columns = prepare_data_for_training(processed_data, feature_info)

In [ ]:
# Initialize the attention model builder
n_features = len(feature_columns)
attention_builder = FinancialAttentionModel(
    n_features=n_features,
    n_heads=8,
    d_model=128,
    dropout_rate=0.1
)

print(f"Building attention models for {n_features} features")

In [ ]:
# 1. Build and train Savings Achievement Classification Model
print("=" * 50)
print("TRAINING SAVINGS ACHIEVEMENT MODEL")
print("=" * 50)

# Build model
savings_model = attention_builder.build_classification_model("savings_achievement")
savings_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print(savings_model.summary())

# Prepare data
y_savings = targets['can_achieve_savings']
X_train, X_test, y_train, y_test = train_test_split(X, y_savings, test_size=0.2, random_state=42, stratify=y_savings)

# Training callbacks
callbacks_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    callbacks.ModelCheckpoint('trained_model/best_savings_model.keras', save_best_only=True, monitor='val_accuracy')
]

# Train model
history_savings = savings_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=callbacks_list,
    verbose=1
)

# Evaluate
y_pred = (savings_model.predict(X_test) > 0.5).astype(int)
print("\nSavings Achievement Classification Results:")
print(classification_report(y_test, y_pred))

In [ ]:
# 2. Build and train Savings Amount Regression Model
print("=" * 50)
print("TRAINING SAVINGS AMOUNT REGRESSION MODEL")
print("=" * 50)

# Build regression model
amount_model = attention_builder.build_regression_model("savings_amount")
amount_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mse']
)

# Prepare data
y_amount = targets['savings_amount']
X_train_amt, X_test_amt, y_train_amt, y_test_amt = train_test_split(X, y_amount, test_size=0.2, random_state=42)

# Training callbacks
callbacks_regression = [
    callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    callbacks.ModelCheckpoint('trained_model/best_amount_model.keras', save_best_only=True, monitor='val_mae')
]

# Train model
history_amount = amount_model.fit(
    X_train_amt, y_train_amt,
    validation_data=(X_test_amt, y_test_amt),
    epochs=20,
    batch_size=32,
    callbacks=callbacks_regression,
    verbose=1
)

# Evaluate
y_pred_amt = amount_model.predict(X_test_amt)
mae = np.mean(np.abs(y_test_amt - y_pred_amt.flatten()))
mse = np.mean((y_test_amt - y_pred_amt.flatten()) ** 2)
print(f"\nSavings Amount Regression Results:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {np.sqrt(mse):.2f}")

In [ ]:
# 3. Build and train Multi-task Model - FIXED VERSION
print("=" * 50)
print("TRAINING MULTI-TASK MODEL")
print("=" * 50)

# Build multi-task model
multi_task_model = attention_builder.build_multi_task_model()
multi_task_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'can_achieve_savings': 'binary_crossentropy',
        'savings_amount': 'mse',
        'financial_risk': 'binary_crossentropy'
    },
    loss_weights={
        'can_achieve_savings': 1.0,
        'savings_amount': 0.001,  # Scale down due to different magnitude
        'financial_risk': 1.0
    },
    metrics={
        'can_achieve_savings': ['accuracy'],
        'savings_amount': ['mae'],
        'financial_risk': ['accuracy']
    }
)

print(multi_task_model.summary())

# Prepare multi-task data - FIXED APPROACH
# Create combined target array for splitting
y_combined = np.column_stack([
    targets['can_achieve_savings'],
    targets['savings_amount'],
    targets['financial_risk']
])

# Split the data
X_train_multi, X_test_multi, y_train_combined, y_test_combined = train_test_split(
    X, y_combined, test_size=0.2, random_state=42
)

# Convert back to dictionary format for multi-output training
y_train_multi_dict = {
    'can_achieve_savings': y_train_combined[:, 0],
    'savings_amount': y_train_combined[:, 1],
    'financial_risk': y_train_combined[:, 2]
}

y_test_multi_dict = {
    'can_achieve_savings': y_test_combined[:, 0],
    'savings_amount': y_test_combined[:, 1],
    'financial_risk': y_test_combined[:, 2]
}

print(f"Multi-task training data shapes:")
print(f"X_train_multi: {X_train_multi.shape}")
print(f"y_train shapes: {[y_train_multi_dict[key].shape for key in y_train_multi_dict.keys()]}")

# Training callbacks
callbacks_multi = [
    callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6),
    callbacks.ModelCheckpoint('trained_model/best_multi_task_model.keras', save_best_only=True, monitor='val_loss')
]

# Train multi-task model
history_multi = multi_task_model.fit(
    X_train_multi, y_train_multi_dict,
    validation_data=(X_test_multi, y_test_multi_dict),
    epochs=20,
    batch_size=32,
    callbacks=callbacks_multi,
    verbose=1
)

# Evaluate multi-task model
print("\nMulti-task Model Evaluation:")
predictions_multi = multi_task_model.predict(X_test_multi)

# Evaluate each task
# 1. Savings Achievement Classification
savings_pred = (predictions_multi[0] > 0.5).astype(int).flatten()
print("\nSavings Achievement Task:")
print(classification_report(y_test_combined[:, 0], savings_pred))

# 2. Savings Amount Regression
amount_pred = predictions_multi[1].flatten()
mae_multi = np.mean(np.abs(y_test_combined[:, 1] - amount_pred))
mse_multi = np.mean((y_test_combined[:, 1] - amount_pred) ** 2)
print(f"\nSavings Amount Task:")
print(f"MAE: {mae_multi:.2f}")
print(f"MSE: {mse_multi:.2f}")
print(f"RMSE: {np.sqrt(mse_multi):.2f}")

# 3. Financial Risk Classification
risk_pred = (predictions_multi[2] > 0.5).astype(int).flatten()
print("\nFinancial Risk Task:")
print(classification_report(y_test_combined[:, 2], risk_pred))

In [ ]:
# Visualization and Analysis
def plot_training_history(history, title, save_name=None):
    """Plot training history"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot loss
    axes[0].plot(history.history['loss'], label='Training Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title(f'{title} - Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Plot main metric
    metric_keys = [k for k in history.history.keys() if 'accuracy' in k or 'mae' in k]
    if metric_keys:
        main_metric = metric_keys[0]
        val_metric = f'val_{main_metric}'
        if val_metric in history.history:
            axes[1].plot(history.history[main_metric], label=f'Training {main_metric}')
            axes[1].plot(history.history[val_metric], label=f'Validation {main_metric}')
            axes[1].set_title(f'{title} - {main_metric}')
            axes[1].set_xlabel('Epochs')
            axes[1].set_ylabel(main_metric)
            axes[1].legend()
            axes[1].grid(True)
    
    plt.tight_layout()
    if save_name:
        plt.savefig(f'{save_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot training histories
plot_training_history(history_savings, "Savings Achievement Model", "savings_training")
plot_training_history(history_amount, "Savings Amount Model", "amount_training")
plot_training_history(history_multi, "Multi-Task Model", "multi_task_training")

In [ ]:
# Model Evaluation and Feature Importance Analysis
def analyze_attention_weights(model, X_sample, feature_names, sample_idx=0):
    """Analyze attention weights for interpretability"""
    # Get attention layer outputs
    attention_layer = None
    for layer in model.layers:
        if 'attention' in layer.name and 'mha' in layer.name:
            attention_layer = layer
            break
    
    if attention_layer:
        # Create a model that outputs attention weights
        attention_model = Model(
            inputs=model.input,
            outputs=attention_layer.output
        )
        
        # Get attention weights for a sample
        sample = X_sample[sample_idx:sample_idx+1]
        attention_output = attention_model.predict(sample)
        
        return attention_output
    
    return None

# Save models and create model summary
model_summary = {
    "models_created": {
        "savings_achievement_model": {
            "type": "binary_classification",
            "architecture": "attention",
            "input_features": n_features,
            "saved_as": "trained_model/best_savings_model.keras"
        },
        "savings_amount_model": {
            "type": "regression", 
            "architecture": "attention",
            "input_features": n_features,
            "saved_as": "trained_model/best_amount_model.keras"
        },
        "multi_task_model": {
            "type": "multi_task",
            "tasks": ["can_achieve_savings", "savings_amount", "financial_risk"],
            "architecture": "attention",
            "input_features": n_features,
            "saved_as": "trained_model/best_multi_task_model.keras"
        }
    },
    "feature_info": {
        "total_features": n_features,
        "feature_names": feature_columns,
        "numerical_features": len(feature_info['numerical_features']),
        "categorical_features": len(feature_info['categorical_features'])
    }
}

# Save model summary
with open('../model/attention_models_summary.json', 'w') as f:
    json.dump(model_summary, f, indent=2)

print("Attention models training complete!")
print(f"Models saved:")
print("1. trained_model/best_savings_model.keras - Savings achievement classification")
print("2. trained_model/best_amount_model.keras - Savings amount regression") 
print("3. trained_model/best_multi_task_model.keras - Multi-task model")
print("4. ../model/attention_models_summary.json - Model metadata")

In [1]:
# Create prediction functions for real-world usage - FIXED VERSION
def create_prediction_pipeline():
    """Create easy-to-use prediction functions"""
    
    # Define custom objects that might be needed for loading
    custom_objects_for_load = {'Cast': tf.cast}
    
    # Local variables for loaded models
    loaded_savings_model = None
    loaded_amount_model = None
    loaded_multi_task_model = None

    try:
        # Try to load SavedModel format first
        try:
            print("Attempting to load models from SavedModel format...")
            loaded_savings_model = tf.keras.models.load_model('trained_model/best_savings_model', custom_objects=custom_objects_for_load, compile=False)
            loaded_amount_model = tf.keras.models.load_model('trained_model/best_amount_model', custom_objects=custom_objects_for_load, compile=False)
            loaded_multi_task_model = tf.keras.models.load_model('trained_model/best_multi_task_model', custom_objects=custom_objects_for_load, compile=False)
            print("Loaded models from SavedModel format.")
        except Exception as e_sm:
            print(f"Failed to load from SavedModel format: {e_sm}")
            print("Attempting to load models from .keras format...")
            # Fallback to .keras format with custom objects
            loaded_savings_model = tf.keras.models.load_model('trained_model/best_savings_model.keras', 
                                                     custom_objects=custom_objects_for_load, 
                                                     compile=False)
            loaded_amount_model = tf.keras.models.load_model('trained_model/best_amount_model.keras', 
                                                    custom_objects=custom_objects_for_load, 
                                                    compile=False)
            loaded_multi_task_model = tf.keras.models.load_model('trained_model/best_multi_task_model.keras', 
                                                        custom_objects=custom_objects_for_load, 
                                                        compile=False)
            print("Loaded models from .keras format.")
    except Exception as e_load_all:
        print(f"Could not load saved models: {e_load_all}")
        print("Using models from current session (if they were trained and are globally available)...")
        # If loading fails, the predict functions will rely on Python's scoping rules
        # to find savings_model, amount_model, multi_task_model in the global scope
        # (assuming they were trained earlier in the notebook).
        # To make this explicit for the predict functions defined below,
        # we assign the global models to the local variables.
        # This assumes the global variables have these exact names.
        global savings_model, amount_model, multi_task_model
        loaded_savings_model = savings_model
        loaded_amount_model = amount_model
        loaded_multi_task_model = multi_task_model
        
    # Assign to the names expected by the inner prediction functions
    # These will be captured by the closures of the prediction functions
    # This step is crucial: make the successfully loaded (or fallback global) models
    # available to the predict_... functions under the names they expect.
    savings_model_to_use = loaded_savings_model
    amount_model_to_use = loaded_amount_model
    multi_task_model_to_use = loaded_multi_task_model

    if None in [savings_model_to_use, amount_model_to_use, multi_task_model_to_use]:
        print("Warning: One or more models could not be loaded and are not available from the global session. Prediction functions may fail.")
        # Optionally, raise an error here if models are essential:
        # raise RuntimeError("Failed to initialize prediction pipeline: Models not available.")

    def predict_savings_achievement(features):
        """Predict if user can achieve their savings goal"""
        if savings_model_to_use is None:
            raise RuntimeError("Savings achievement model is not available.")
        prediction = savings_model_to_use.predict(features.reshape(1, -1), verbose=0)
        return {
            'can_achieve_savings': bool(prediction[0][0] > 0.5),
            'confidence': float(prediction[0][0])
        }
    
    def predict_optimal_savings_amount(features):
        """Predict optimal savings amount"""
        if amount_model_to_use is None:
            raise RuntimeError("Savings amount model is not available.")
        prediction = amount_model_to_use.predict(features.reshape(1, -1), verbose=0)
        return {
            'recommended_savings': float(prediction[0][0])
        }
    
    def predict_comprehensive_financial_profile(features):
        """Get comprehensive financial predictions"""
        if multi_task_model_to_use is None:
            raise RuntimeError("Multi-task model is not available.")
        predictions = multi_task_model_to_use.predict(features.reshape(1, -1), verbose=0)
        return {
            'can_achieve_savings': bool(predictions[0][0][0] > 0.5),
            'savings_confidence': float(predictions[0][0][0]),
            'recommended_savings_amount': float(predictions[1][0][0]),
            'financial_risk': bool(predictions[2][0][0] > 0.5),
            'risk_score': float(predictions[2][0][0])
        }
    
    return predict_savings_achievement, predict_optimal_savings_amount, predict_comprehensive_financial_profile

# Save models in both formats for better compatibility
print("Saving models in multiple formats...")

# Save in SavedModel format (recommended)
try:
    predict_savings, predict_amount, predict_comprehensive = create_prediction_pipeline()
    
    # Test with a sample
    sample_features = X_test[0]
    print("\nSample Predictions:")
    print("Savings Achievement:", predict_savings(sample_features))
    print("Optimal Amount:", predict_amount(sample_features))
    print("Comprehensive Profile:", predict_comprehensive(sample_features))
    
    print("\n" + "="*50)
    print("ATTENTION MODELS READY FOR DEPLOYMENT!")
    print("="*50)
    
except Exception as e:
    print(f"Error in prediction pipeline: {e}")
    print("Models are trained but prediction pipeline needs adjustment")

Saving models in multiple formats...
Error in prediction pipeline: name 'tf' is not defined
Models are trained but prediction pipeline needs adjustment


In [2]:
import tensorflow as tf
import numpy as np

def test_saved_models(sample_features):
    """Test all three saved models with a sample feature vector."""
    # Load models (try .keras format for simplicity)
    savings_model = tf.keras.models.load_model('trained_model/best_savings_model.keras', compile=False)
    amount_model = tf.keras.models.load_model('trained_model/best_amount_model.keras', compile=False)
    multi_task_model = tf.keras.models.load_model('trained_model/best_multi_task_model.keras', compile=False)

    # Ensure input is numpy array and correct shape
    features = np.array(sample_features).reshape(1, -1)

    # 1. Savings Achievement Prediction
    pred_savings = savings_model.predict(features, verbose=0)
    savings_result = {
        'can_achieve_savings': bool(pred_savings[0][0] > 0.5),
        'confidence': float(pred_savings[0][0])
    }

    # 2. Savings Amount Prediction
    pred_amount = amount_model.predict(features, verbose=0)
    amount_result = {
        'recommended_savings': float(pred_amount[0][0])
    }

    # 3. Multi-task Model Prediction
    pred_multi = multi_task_model.predict(features, verbose=0)
    multi_result = {
        'can_achieve_savings': bool(pred_multi[0][0][0] > 0.5),
        'savings_confidence': float(pred_multi[0][0][0]),
        'recommended_savings_amount': float(pred_multi[1][0][0]),
        'financial_risk': bool(pred_multi[2][0][0] > 0.5),
        'risk_score': float(pred_multi[2][0][0])
    }

    print("Test Results for Saved Models:")
    print("Savings Achievement:", savings_result)
    print("Optimal Amount:", amount_result)
    print("Comprehensive Profile:", multi_result)
    return savings_result, amount_result, multi_result

# Example usage:

In [20]:
test_saved_models(X_test[0])

Test Results for Saved Models:
Savings Achievement: {'can_achieve_savings': True, 'confidence': 0.9999999403953552}
Optimal Amount: {'recommended_savings': 843.02392578125}
Comprehensive Profile: {'can_achieve_savings': True, 'savings_confidence': 0.9953286051750183, 'recommended_savings_amount': 1025.1810302734375, 'financial_risk': True, 'risk_score': 0.5527254343032837}


({'can_achieve_savings': True, 'confidence': 0.9999999403953552},
 {'recommended_savings': 843.02392578125},
 {'can_achieve_savings': True,
  'savings_confidence': 0.9953286051750183,
  'recommended_savings_amount': 1025.1810302734375,
  'financial_risk': True,
  'risk_score': 0.5527254343032837})